In [39]:
import pandas as pd
import numpy as np
import re
from sklearn.cross_decomposition import PLSRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score

import utils.dev_config as dev_conf
import utils.preprocessing as prep

In [2]:
dirs = dev_conf.get_dev_directories("../dev_paths.txt")
unified_dsets = ["unified_cervical_data", "unified_uterine_data", "unified_uterine_endometrial_data"]
matrisome_list = f"{dirs.data_dir}/matrisome/matrisome_hs_masterlist.tsv"

In [3]:
i = 0

In [4]:
matrisome_df = prep.load_matrisome_df(matrisome_list)

# Load and filter survival data

In [13]:
event_code = {"Alive": 0, "Dead": 1}
covariate_cols = ["figo_stage", "age_at_diagnosis", "race", "ethnicity"]
dep_cols = ["vital_status", "survival_time"]
cat_cols = ["race", "ethnicity", "figo_chr"]
survival_df = prep.load_survival_df(f"{dirs.data_dir}/{unified_dsets[i]}/survival_data.tsv", event_code)

In [15]:
filtered_survival_df = (
    prep.decode_figo_stage(survival_df[["sample_name"] + dep_cols + covariate_cols].dropna(), to="c")
        .query("vital_status == 1")
        .drop(["vital_status"], axis=1)
        .reset_index(drop = True)
)

print(filtered_survival_df.shape)
filtered_survival_df.head()

(66, 6)


,sample_name,survival_time,age_at_diagnosis,race,ethnicity,figo_chr
0,TCGA-C5-A1BF-01B-11R-A13Y-07,570,16975.0,white,not reported,figo_stage_1
1,TCGA-C5-A8YT-01A-11R-A37O-07,633,13253.0,white,not hispanic or latino,figo_stage_1
2,TCGA-C5-A1BE-01B-11R-A13Y-07,2094,23727.0,white,not reported,figo_stage_1
3,TCGA-C5-A8XH-01A-11R-A37O-07,1394,14444.0,white,not hispanic or latino,figo_stage_1
4,TCGA-DS-A7WF-01A-11R-A352-07,492,15319.0,not reported,hispanic or latino,figo_stage_1


# Load normalized matrisome count data

In [16]:
norm_matrisome_counts_df = pd.read_csv(f"{dirs.data_dir}/{unified_dsets[i]}/norm_matrisome_counts.tsv", sep='\t')
norm_filtered_matrisome_counts_t_df = prep.transpose_df(
    norm_matrisome_counts_df[["geneID"] + list(filtered_survival_df.sample_name)], "geneID", "sample_name"
)
print(norm_filtered_matrisome_counts_t_df.shape)
norm_filtered_matrisome_counts_t_df.head()

(66, 1009)


,sample_name,PGF,TIMP4,C1QTNF6,TNC,PRL,OGN,C1QL3,FGB,NDNF,...,PIK3IP1,C1QTNF2,PCSK5,ANXA1,HGF,VWA2,FGF3,POSTN,NTF3,S100A6
0,TCGA-C5-A1BF-01B-11R-A13Y-07,10.800637,6.228003,11.669331,13.002928,5.063964,4.869744,5.063964,8.834522,6.410767,...,9.013453,8.190325,9.503647,14.077995,6.569726,7.315604,4.602649,12.062300,5.649441,16.558407
1,TCGA-C5-A8YT-01A-11R-A37O-07,7.830611,5.733875,12.445548,13.765468,5.455125,13.049104,5.146455,5.074289,10.569544,...,9.453187,6.398956,12.288955,13.396332,10.228758,8.542025,4.602649,11.765396,5.318924,13.556322
2,TCGA-C5-A1BE-01B-11R-A13Y-07,10.642039,5.348449,8.945220,13.419225,4.602649,5.867905,5.646251,4.602649,5.673617,...,11.696884,6.384980,9.774029,15.381944,6.205261,7.163053,4.957257,10.113788,5.213815,15.564969
3,TCGA-C5-A8XH-01A-11R-A37O-07,9.633752,5.908552,11.672191,13.863766,4.602649,5.339887,5.702581,5.389600,5.634507,...,11.148165,7.526140,8.561116,14.404419,6.462928,6.102570,4.602649,9.104547,5.088257,15.197350
4,TCGA-DS-A7WF-01A-11R-A352-07,13.292479,5.620704,12.867887,16.646519,4.602649,10.377267,6.177498,4.602649,5.198452,...,10.809104,7.877841,6.615459,13.894278,7.058159,10.809104,4.602649,15.271686,6.519692,16.820793


# Join survival and count data

In [26]:
joined_df = (
    pd.merge(filtered_survival_df, norm_filtered_matrisome_counts_t_df, on="sample_name")
        .set_index("sample_name")
        .pipe(pd.get_dummies, cat_cols)
)
print(joined_df.shape)
joined_df.head()

(66, 1023)


,survival_time,age_at_diagnosis,PGF,TIMP4,C1QTNF6,TNC,PRL,OGN,C1QL3,FGB,...,race_native hawaiian or other pacific islander,race_not reported,race_white,ethnicity_hispanic or latino,ethnicity_not hispanic or latino,ethnicity_not reported,figo_chr_figo_stage_1,figo_chr_figo_stage_2,figo_chr_figo_stage_3,figo_chr_figo_stage_4
sample_name,,,,,,,,,,,,,,,,,,,,,
TCGA-C5-A1BF-01B-11R-A13Y-07,570,16975.0,10.800637,6.228003,11.669331,13.002928,5.063964,4.869744,5.063964,8.834522,...,0,0,1,0,0,1,1,0,0,0
TCGA-C5-A8YT-01A-11R-A37O-07,633,13253.0,7.830611,5.733875,12.445548,13.765468,5.455125,13.049104,5.146455,5.074289,...,0,0,1,0,1,0,1,0,0,0
TCGA-C5-A1BE-01B-11R-A13Y-07,2094,23727.0,10.642039,5.348449,8.945220,13.419225,4.602649,5.867905,5.646251,4.602649,...,0,0,1,0,0,1,1,0,0,0
TCGA-C5-A8XH-01A-11R-A37O-07,1394,14444.0,9.633752,5.908552,11.672191,13.863766,4.602649,5.339887,5.702581,5.389600,...,0,0,1,0,1,0,1,0,0,0
TCGA-DS-A7WF-01A-11R-A352-07,492,15319.0,13.292479,5.620704,12.867887,16.646519,4.602649,10.377267,6.177498,4.602649,...,0,1,0,1,0,0,1,0,0,0


# PLSR

In [35]:
X = joined_df.iloc[:, 1:].values
# Y must be a matrix
Y = joined_df.iloc[:, 1].values[:, np.newaxis]

In [37]:
plsr_model = PLSRegression()
plsr_pipeline = make_pipeline(StandardScaler(), plsr_model)